In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load your dataset
data = pd.read_excel('Line Listing(dxjZy).xlsx')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
selected_columns = ['Case ID', 'Suspect Product Active Ingredients', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']

df = data[selected_columns]

df.head()
print(df)

In [ ]:
df['Suspect Product Active Ingredients'] = df['Suspect Product Active Ingredients'].str.split(';')
df_split = df.explode('Suspect Product Active Ingredients', ignore_index=True)

df_split.head()

In [ ]:
count = df_split['Serious'].value_counts()

print(count)

In [ ]:
print(df_split.dtypes)

In [ ]:
df_multi = pd.get_dummies(df_split, columns=['Suspect Product Active Ingredients'], prefix='', prefix_sep='')

df_multi = df_multi.groupby('Case ID').max().reset_index()

# Columns to exclude from conversion
columns_to_exclude = ['Case ID', 'Suspect Product Active Ingredients', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']

# Get the columns to convert by excluding the specified ones
columns_to_convert = [col for col in df_multi.columns if col not in columns_to_exclude]

# Convert only the desired columns to 0/1
df_multi[columns_to_convert] = df_multi[columns_to_convert].astype(int)

df_final = df_multi
print(df_multi.columns)
print(df_final)

df_final.head()

In [ ]:
print(df_multi.columns)
#df_encoded = df_split
df_encoded = df_multi.copy()
print(df_encoded.columns)
#print(df_split['Patient Age'])
df_encoded['Patient Age'] = df_encoded['Patient Age'].astype(str)
df_encoded['Patient Age'] = df_encoded['Patient Age'].str.replace(r'\D+', '', regex=True)
df_encoded['Patient Age'] = pd.to_numeric(df_encoded['Patient Age'], errors='coerce')  # Converts to numeric, sets invalid values to NaN

df_encoded['Patient Weight'] = df_encoded['Patient Weight'].replace('Not Specified', "0 KG")
df_encoded['Patient Weight'] = df_encoded['Patient Weight'].astype(str)
#df_encoded['Patient Weight'] = df_encoded['Patient Weight'].str.replace(r'\D+', '', regex=True)
df_encoded['Patient Weight'] = df_encoded['Patient Weight'].str.replace(r'[^\d.]', '', regex=True)
df_encoded['Patient Weight'] = pd.to_numeric(df_encoded['Patient Weight'], errors='coerce')

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

#df_encoded['Suspect Product Active Ingredients'] = label_encoder.fit_transform(df_encoded['Suspect Product Active Ingredients'])
df_encoded['Sex'] = label_encoder.fit_transform(df_encoded['Sex'])
df_encoded['Serious'] = label_encoder.fit_transform(df_encoded['Serious'])
print(df_encoded.isnull().sum())
df_encoded.dropna(inplace=True)
print(df_encoded.isnull().sum())
df_encoded.head()
print(df_encoded)

In [ ]:
count = df_encoded['Serious'].value_counts()

print(count)

In [ ]:
feature_cols = ['Suspect Product Active Ingredients', 'Sex', 'Patient Age', 'Patient Weight']

# Columns to exclude from conversion
columns_to_exclude = ['Case ID', 'Suspect Product Active Ingredients']

# Get the columns to convert by excluding the specified ones
feature_cols = [col for col in df_encoded.columns if col not in columns_to_exclude]
print(feature_cols)

X = df_encoded[feature_cols]
y = df_encoded.Serious
#X = df_split[feature_cols]
#Y = df_split.Serious

In [ ]:
X.head()

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Adjust activation for specific reaction types
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)



In [ ]:
# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")